Source api: [OPEN-METEO](https://open-meteo.com/)

In [2]:
import requests as req
import pandas as pd
import matplotlib.pyplot as plt
import requests_cache
from retry_requests import retry
import openmeteo_requests

In [3]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [4]:
weather_folder_path = './data/weather'

In [5]:
years = [('2017-01-01', '2017-12-31'),
         ('2018-01-01', '2018-12-31'),
         ('2019-01-01', '2019-12-31'),
         ('2020-01-01', '2020-12-31'),
         ('2021-01-01', '2021-12-31'),
         ('2022-01-01', '2022-12-31'),
         ('2023-01-01', '2023-12-31'),
         ('2024-01-01', '2024-03-24')]

In [6]:
features = ['temperature_2m',
           'relative_humidity_2m',
           'pressure_msl',
           'surface_pressure',
           'precipitation',
           'rain',
           'cloud_cover',
           'cloud_cover_low',
           'cloud_cover_mid',
           'cloud_cover_high',
           'shortwave_radiation',
           'direct_radiation',
           'diffuse_radiation',
           'sunshine_duration',
           'wind_speed_10m',
           'wind_speed_100m',
           'wind_direction_10m',
           'wind_direction_100m',
           'wind_gusts_10m',]

In [7]:
bp = {'lat': 47.4925, 'long': 19.051389, 'name' : 'BP'}

In [8]:
url = "https://archive-api.open-meteo.com/v1/archive"

In [9]:
def get_data(location, start_date, end_date, features):
    features_str = ','.join(features)
    features_str

    params = {
        "latitude": location.get('lat'),
        "longitude": location.get('long'),
        "start_date": start_date,
        "end_date": end_date,
        "hourly":  features_str
    }
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    hourly = response.Hourly()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
        )}
    
    for f in features:
        hourly_data[f"{f}_{location.get('name')}"] = hourly.Variables(features.index(f)).ValuesAsNumpy()

    df = pd.DataFrame(hourly_data)
    return df


In [10]:
location = bp

In [11]:
for year in years:
    df = get_data(location, year[0], year[1], features)
    df_final = df if year == years[0] else pd.concat([df_final, df])
df_final.to_csv(f'{weather_folder_path}/weather_{location.get('name')}_{years[0][0],years[-1][-1]}.csv', index = False)

In [12]:
df_final

,date,temperature_2m_BP,relative_humidity_2m_BP,pressure_msl_BP,surface_pressure_BP,precipitation_BP,rain_BP,cloud_cover_BP,cloud_cover_low_BP,cloud_cover_mid_BP,cloud_cover_high_BP,shortwave_radiation_BP,direct_radiation_BP,diffuse_radiation_BP,sunshine_duration_BP,wind_speed_10m_BP,wind_speed_100m_BP,wind_direction_10m_BP,wind_direction_100m_BP,wind_gusts_10m_BP
0,2017-01-01 00:00:00+00:00,-3.579,95.965530,1029.599976,1014.724121,0.0,0.0,100.000000,96.0,0.0,94.0,0.0,0.0,0.0,0.0,6.379216,9.178235,286.389618,295.559906,12.599999
1,2017-01-01 01:00:00+00:00,-4.429,95.937294,1029.400024,1014.480225,0.0,0.0,65.400002,44.0,0.0,86.0,0.0,0.0,0.0,0.0,5.091168,13.104198,315.000092,307.184784,10.080000
2,2017-01-01 02:00:00+00:00,-6.079,95.514832,1028.699951,1013.699158,0.0,0.0,61.799999,41.0,0.0,83.0,0.0,0.0,0.0,0.0,7.200000,11.720751,323.130005,317.489594,10.799999
3,2017-01-01 03:00:00+00:00,-5.729,95.893539,1028.300049,1013.324524,0.0,0.0,54.599998,32.0,0.0,86.0,0.0,0.0,0.0,0.0,5.860375,9.957108,317.489594,319.398773,10.799999
4,2017-01-01 04:00:00+00:00,-5.279,99.244804,1027.900024,1012.955200,0.0,0.0,44.099998,47.0,0.0,6.0,0.0,0.0,0.0,0.0,2.741678,6.792466,336.801392,327.994659,9.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,2024-03-24 19:00:00+00:00,5.421,80.713318,1007.200012,993.114014,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.830519,18.079027,251.564957,257.347443,12.959999
2012,2024-03-24 20:00:00+00:00,4.171,88.378944,1007.099976,992.952393,0.0,0.0,1.800000,2.0,0.0,0.0,0.0,0.0,0.0,0.0,6.830519,19.353140,251.564957,260.362518,12.240000
2013,2024-03-24 21:00:00+00:00,3.721,90.892494,1007.000000,992.831055,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.072878,19.813087,255.256393,267.917480,12.959999
2014,2024-03-24 22:00:00+00:00,3.471,90.874847,1006.799988,992.621033,0.0,0.0,11.400001,0.0,11.0,16.0,0.0,0.0,0.0,0.0,8.350138,22.505037,262.568695,277.352295,14.400000
